In [2]:
import os
import sys
import pandas as pd
from pykrx import stock
from datetime import datetime

# --- 프로젝트 루트 경로 설정 ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
    print(f"Added project root to sys.path: {PROJECT_ROOT}")
# ---------------------------------

# --- 불러올 기업 리스트 (6개 종목) ---
TICKER_LIST = ["329180", "042660", "009540", "010140", "443060", "010620"]
# (로그 확인용 종목명 매핑)
TICKER_NAMES = {
    "329180": "HD현대마린솔루션",
    "042660": "HD한국조선해양",
    "009540": "HD현대중공업",
    "010140": "삼성중공업",
    "443060": "한화오션",
    "010620": "현대미포조선"
}

# --- 저장할 기본 디렉토리 ---
SAVE_DIR = os.path.join(PROJECT_ROOT, "data", "silver", "prices_daily")
# 디렉토리 미리 생성 (루프 밖에서 한 번만)
os.makedirs(SAVE_DIR, exist_ok=True)
print(f"--- Base Save Directory: {SAVE_DIR} ---")

# --- 데이터 기간 설정 (2019년 ~ 2024년) ---
START_DATE = "20190101"
END_DATE = "20241231" 

print(f"--- Data Period Set: {START_DATE} ~ {END_DATE} ---")

# --- 6개 종목 순환 시작 ---
for ticker in TICKER_LIST:
    ticker_name = TICKER_NAMES.get(ticker, "Unknown Ticker")
    SAVE_FILE_PATH = os.path.join(SAVE_DIR, f"{ticker}.csv")
    
    print(f"\n--- Processing: {ticker} ({ticker_name}) ---")
    print(f"Target file: {SAVE_FILE_PATH}")

    try:
        # 1. pykrx를 사용해 데이터 조회
        print(f"Fetching data for {ticker}...")
        df = stock.get_market_ohlcv(START_DATE, END_DATE, ticker)
        
        if df.empty:
            # (참고: HD현대마린솔루션(329180)은 2024년 상장되어 2019년 데이터가 없는 것이 정상)
            print(f"Warning: No data found for ticker {ticker} in the specified period. Skipping.")
            continue # 다음 티커로 넘어감

        # 2. 데이터프레임 처리
        df = df.reset_index()
        
        # 컬럼 이름 변경 (한글 -> 영문)
        df = df.rename(columns={
            '날짜': 'date',
            '시가': 'open',
            '고가': 'high',
            '저가': 'low',
            '종가': 'close',
            '거래량': 'trading_volume'
        })
        
        # 'date' 컬럼을 YYYY-MM-DD 형식의 문자열로 변환
        df['date'] = df['date'].dt.strftime('%Y-%m-%d')
        
        # 필요한 컬럼만 선택
        required_columns = ['date', 'open', 'high', 'low', 'close', 'trading_volume']
        
        # (혹시 모를 pykrx 구버전 대비)
        # 이미 영문(close)이 있다면 rename은 무시되므로 안전합니다.
        
        # 실제 존재하는 컬럼만 필터링 (에러 방지)
        final_columns = [col for col in required_columns if col in df.columns]
        final_df = df[final_columns]

        # 3. CSV 파일로 저장
        final_df.to_csv(SAVE_FILE_PATH, index=False)
        
        print(f"--- Success! ---")
        print(f"Saved {len(final_df)} rows for {ticker} to:")
        print(f"{SAVE_FILE_PATH}")
        print(final_df.head(3)) # 3줄만 미리보기

    except Exception as e:
        print(f"\nAn error occurred while processing ticker {ticker}: {e}")
        print("Skipping to the next ticker.")

print("\n--- All tickers processed. ---")

--- Base Save Directory: /workspace/ship-ai/data/silver/prices_daily ---
--- Data Period Set: 20190101 ~ 20241231 ---

--- Processing: 329180 (HD현대마린솔루션) ---
Target file: /workspace/ship-ai/data/silver/prices_daily/329180.csv
Fetching data for 329180...
--- Success! ---
Saved 805 rows for 329180 to:
/workspace/ship-ai/data/silver/prices_daily/329180.csv
         date    open    high    low   close  trading_volume
0  2021-09-17  111000  135000  91000  111500        17373529
1  2021-09-23  112000  113000  99000   99100         2914293
2  2021-09-24  100000  102000  95500   99200         2262592

--- Processing: 042660 (HD한국조선해양) ---
Target file: /workspace/ship-ai/data/silver/prices_daily/042660.csv
Fetching data for 042660...
--- Success! ---
Saved 1477 rows for 042660 to:
/workspace/ship-ai/data/silver/prices_daily/042660.csv
         date   open   high    low  close  trading_volume
0  2019-01-02  30428  30650  29406  29584          271568
1  2019-01-03  29939  30383  29495  29762     